In [1]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

In [2]:
import findspark
findspark.init()

In [3]:
spark = SparkSession.builder.getOrCreate()


In [4]:
data = spark.read.csv('../data/ruddit_comments_score.csv', header=True, inferSchema=True, sep = ",", escape = "", multiLine = True)

In [5]:
data.show()

+--------------------+--------------------+------+
|          comment_id|                body| score|
+--------------------+--------------------+------+
|             cza1q49|> The difference ...|-0.083|
|             cza1wdh|"The myth is that...|-0.022|
|             cza23qx|           [deleted]| 0.167|
|             cza2bw8|The assertion is ...|-0.146|
|             cza2iji|You said in the O...|-0.083|
|             cza2jj3|">Men and women a...|  null|
|Edit: Changed 70 ...|              -0.042|  null|
|             cza31e2|> All the wage ga...|-0.021|
|             cza321d|           [deleted]|-0.021|
|             cza336e|           [deleted]| 0.208|
|             cza34dq|           [deleted]|-0.191|
|             cza3500|           [deleted]|-0.229|
|             cza37ue|No, the point is ...|-0.174|
|             cza3802|           [deleted]| 0.021|
|             cza392y|So women are paid...|-0.229|
|             cza3avu|           [deleted]|-0.167|
|             cza3cmh|         

In [6]:
data.printSchema()

root
 |-- comment_id: string (nullable = true)
 |-- body: string (nullable = true)
 |-- score: string (nullable = true)



In [7]:
data = data.dropna()
data = data.filter(data.body != "[deleted]")
data.show()

+----------+--------------------+------+
|comment_id|                body| score|
+----------+--------------------+------+
|   cza1q49|> The difference ...|-0.083|
|   cza1wdh|"The myth is that...|-0.022|
|   cza2bw8|The assertion is ...|-0.146|
|   cza2iji|You said in the O...|-0.083|
|   cza31e2|> All the wage ga...|-0.021|
|   cza37ue|No, the point is ...|-0.174|
|   cza392y|So women are paid...|-0.229|
|   cza3m1b|But obviously tha...|   0.0|
|   cza3r5u|"I think that Hol...| 0.098|
|   cza47sd|"> I don't think ...|-0.083|
|   cza47xu|I don't think the...|-0.062|
|   cza4d2a|> Women are a who...|-0.062|
|   cza4gsv|"The gist of my p...|-0.021|
|   cza4ldq|Biological differ...|-0.083|
|   cza5maz|> It's the differ...|-0.188|
|   cza6q74|>The fact of the ...| 0.083|
|   cza6wrd|Well, if your wif...|-0.104|
|   cza76eq|Women have not sp...| 0.188|
|   cza79u4|Doesn't it also m...| 0.175|
|   cza7gpu|So you do believe...|   0.0|
+----------+--------------------+------+
only showing top

In [8]:
data.collect()[1][1]

'"The myth is that the ""gap"" is entirely based on the sex of the  person. "'

In [13]:
data = data.toDF()

In [14]:
data = data.rdd.map(lambda x: (x[0],x[1][1:] if (x[1][0] == '"') else x[1],x[2]))
data = data.toDF()
data = data.rdd.map(lambda x: (x[0],x[1][:-1] if (x[1][-1] == '"') else x[1],x[2]))
data = data.toDF()
data = data.rdd.map(lambda x: (x[0],x[1][1:] if (x[1][0] == '>') else x[1],x[2]))
data = data.toDF()
data = data.rdd.map(lambda x: (x[0],x[1][1:] if (x[1][0] == ' ') else x[1],x[2]))
data = data.toDF()
data = data.rdd.map(lambda x: (x[0],x[1][:-1] if (x[1][-1] == ' ') else x[1],x[2]))
data = data.toDF()
data = data.rdd.map(lambda x: (x[0],x[1][:-1] if (x[1][-1] == '.') else x[1],x[2]))
data = data.toDF()
data = data.rdd.map(lambda x: (x[0],x[1].replace('""', '"'),x[2]))
data = data.toDF()
data.show()

+-------+--------------------+------+
|     _1|                  _2|    _3|
+-------+--------------------+------+
|cza1q49|The difference in...|-0.083|
|cza1wdh|The myth is that ...|-0.022|
|cza2bw8|The assertion is ...|-0.146|
|cza2iji|You said in the O...|-0.083|
|cza31e2|All the wage gap ...|-0.021|
|cza37ue|No, the point is ...|-0.174|
|cza392y|So women are paid...|-0.229|
|cza3m1b|But obviously tha...|   0.0|
|cza3r5u|I think that Holl...| 0.098|
|cza47sd|I don't think man...|-0.083|
|cza47xu|I don't think the...|-0.062|
|cza4d2a|Women are a whole...|-0.062|
|cza4gsv|The gist of my po...|-0.021|
|cza4ldq|Biological differ...|-0.083|
|cza5maz|It's the differen...|-0.188|
|cza6q74|The fact of the m...| 0.083|
|cza6wrd|Well, if your wif...|-0.104|
|cza76eq|Women have not sp...| 0.188|
|cza79u4|Doesn't it also m...| 0.175|
|cza7gpu|So you do believe...|   0.0|
+-------+--------------------+------+
only showing top 20 rows



In [15]:
data.collect()[1][1]

'The myth is that the "gap" is entirely based on the sex of the  person'

In [81]:
data = data.rdd.map(lambda x: (x[0],x[1].replace('""', '"'),x[2]))
data = data.toDF()
data.show()

+-------+--------------------+------+
|     _1|                  _2|    _3|
+-------+--------------------+------+
|cza1q49|The difference in...|-0.083|
|cza1wdh|The myth is that ...|-0.022|
|cza2bw8|The assertion is ...|-0.146|
|cza2iji|You said in the O...|-0.083|
|cza31e2|All the wage gap ...|-0.021|
|cza37ue|No, the point is ...|-0.174|
|cza392y|So women are paid...|-0.229|
|cza3m1b|But obviously tha...|   0.0|
|cza3r5u|I think that Holl...| 0.098|
|cza47sd|I don't think man...|-0.083|
|cza47xu|I don't think the...|-0.062|
|cza4d2a|Women are a whole...|-0.062|
|cza4gsv|The gist of my po...|-0.021|
|cza4ldq|Biological differ...|-0.083|
|cza5maz|It's the differen...|-0.188|
|cza6q74|The fact of the m...| 0.083|
|cza6wrd|Well, if your wif...|-0.104|
|cza76eq|Women have not sp...| 0.188|
|cza79u4|Doesn't it also m...| 0.175|
|cza7gpu|So you do believe...|   0.0|
+-------+--------------------+------+
only showing top 20 rows



In [80]:
a.replace('sfddf', "hola")

'"skfmsklemflksemfklsem"'

In [79]:
a

'"skfmsklemflksemfklsem"'

In [82]:
data.collect()[1][1]

'The myth is that the gap is entirely based on the sex of the  person'

In [47]:
data = data.toDF(["comment_id", "body", "score"])

In [ ]:
data.collect()

In [12]:
b = a.toDF()

In [13]:
b.show()

+-------+--------------------+------+
|     _1|                  _2|    _3|
+-------+--------------------+------+
|cza1q49|> The difference ...|-0.083|
|cza1wdh|The myth is that ...|-0.022|
|cza2bw8|The assertion is ...|-0.146|
|cza2iji|You said in the O...|-0.083|
|cza31e2|> All the wage ga...|-0.021|
|cza37ue|No, the point is ...|-0.174|
|cza392y|So women are paid...|-0.229|
|cza3m1b|But obviously tha...|   0.0|
|cza3r5u|I think that Holl...| 0.098|
|cza47sd|> I don't think m...|-0.083|
|cza47xu|I don't think the...|-0.062|
|cza4d2a|> Women are a who...|-0.062|
|cza4gsv|The gist of my po...|-0.021|
|cza4ldq|Biological differ...|-0.083|
|cza5maz|> It's the differ...|-0.188|
|cza6q74|>The fact of the ...| 0.083|
|cza6wrd|Well, if your wif...|-0.104|
|cza76eq|Women have not sp...| 0.188|
|cza79u4|Doesn't it also m...| 0.175|
|cza7gpu|So you do believe...|   0.0|
+-------+--------------------+------+
only showing top 20 rows



In [ ]:
b.show()

In [113]:
data.select("body").rdd

MapPartitionsRDD[306] at javaToPython at DirectMethodHandleAccessor.java:104

In [168]:
data.rdd.toDF()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 94.0 failed 1 times, most recent failure: Lost task 0.0 in stage 94.0 (TID 94) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.sp

In [ ]:
for row in 

In [88]:
data.select("body").collect()[0].asDict()["body"]

"> The difference in average earnings between men and women can be explained by taking into account relevant factors.\n\nSo it isn't a myth, you just feel that you can explain it."

In [94]:
# data.collect([42])

TypeError: collect() takes 1 positional argument but 2 were given

In [244]:
data.na.drop().show()

+---+-------+--------------------+--------------------+------+
| id|keyword|            location|                text|target|
+---+-------+--------------------+--------------------+------+
| 48| ablaze|          Birmingham|@bbcmtd Wholesale...|     1|
| 49| ablaze|Est. September 20...|We always try to ...|     0|
| 50| ablaze|              AFRICA|#AFRICANBAZE: Bre...|     1|
| 52| ablaze|    Philadelphia, PA|Crying out for mo...|     0|
| 53| ablaze|          London, UK|On plus side LOOK...|     0|
| 54| ablaze|            Pretoria|@PhDSquares #mufc...|     0|
| 55| ablaze|        World Wide!!|INEC Office in Ab...|     1|
| 57| ablaze|      Paranaque City|Ablaze for you Lo...|     0|
| 59| ablaze|      Live On Webcam|Check these out: ...|     0|
| 62| ablaze|           milky way|Had an awesome ti...|     0|
| 66| ablaze|GREENSBORO,NORTH ...|How the West was ...|     1|
| 68| ablaze|      Live On Webcam|Check these out: ...|     0|
| 71| ablaze|            England.|First night with ...|

In [266]:
data.count()

8387

In [268]:
data.tail(3)

[Row(id=10871, keyword=None, location=None, text='M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ', target=1),
 Row(id=10872, keyword=None, location=None, text='Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.', target=1),
 Row(id=10873, keyword=None, location=None, text='The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d', target=1)]

In [243]:
data.show()

+---+-------+--------+--------------------+------+
| id|keyword|location|                text|target|
+---+-------+--------+--------------------+------+
|  1|   null|    null|Our Deeds are the...|     1|
|  4|   null|    null|Forest fire near ...|     1|
|  5|   null|    null|All residents ask...|     1|
|  6|   null|    null|13,000 people rec...|     1|
|  7|   null|    null|Just got sent thi...|     1|
|  8|   null|    null|#RockyFire Update...|     1|
| 10|   null|    null|#flood #disaster ...|     1|
| 13|   null|    null|I'm on top of the...|     1|
| 14|   null|    null|There's an emerge...|     1|
| 15|   null|    null|I'm afraid that t...|     1|
| 16|   null|    null|Three people died...|     1|
| 17|   null|    null|Haha South Tampa ...|     1|
| 18|   null|    null|#raining #floodin...|     1|
| 19|   null|    null|#Flood in Bago My...|     1|
| 20|   null|    null|Damage to school ...|     1|
| 23|   null|    null|      What's up man?|     0|
| 24|   null|    null|       I 